In [ ]:
import glob
import os
import numpy as np
from pcmdi_metrics.graphics import download_archived_results
from pcmdi_metrics.graphics import normalize_by_median
from pcmdi_metrics.graphics import TaylorDiagram
from matplotlib import pyplot as plt
from pcmdi_metrics.graphics import Metrics

In [ ]:
json_list1 = sorted(glob.glob("/global/cfs/cdirs/m4581/lee1043/work/cdat/pmp/ACE_evaluation/mean_climate/output/cmip6/amip/v20251020/*.json"))
for json_file in json_list1:
    print(json_file.split('/')[-1])

In [ ]:
# json_list = sorted(glob.glob(os.path.join(json_dir, '*' + mip + '*' + data_version + '.json')))


In [ ]:
json_list2 = sorted(glob.glob("/global/cfs/cdirs/m4581/lee1043/work/cdat/pmp/ACE_evaluation/mean_climate/output/cmip6/amip/v20250915/*.json"))
for json_file in json_list2:
    print(json_file.split('/')[-1])

In [ ]:
json_list3 = sorted(glob.glob("/global/cfs/cdirs/m4581/lee1043/work/cdat/pmp/ACE_evaluation/mean_climate/output/cmip6/amip/v20240930/*.json"))
for json_file in json_list3:
    print(json_file.split('/')[-1])

In [ ]:
# json_list = json_list1[:]+json_list2[:]+json_list3[3:5]
json_list = json_list1[:]+json_list2[1:2]

In [ ]:
json_list

# ACE and NGCM-evap

In [ ]:
library = Metrics(json_list)
# library = Metrics(json_list1)

In [ ]:
json_list_test_NGCM = sorted(glob.glob(os.path.join('./demo_output/climatology-NGCM/', '*.json')))
for json_file in json_list_test_NGCM:
    print(json_file.split('/')[-1])

In [ ]:
json_list_test_NGCM_evap = sorted(glob.glob(os.path.join('./demo_output/climatology-NGCM-evap/', '*.json')))
for json_file in json_list_test_NGCM_evap:
    print(json_file.split('/')[-1])

In [ ]:
json_list_test = sorted(glob.glob(os.path.join('./demo_output/climatology-ACE/', '*.json')))
for json_file in json_list_test:
    print(json_file.split('/')[-1])

In [ ]:
test_case1 = Metrics(json_list_test_NGCM[0:1])
test_case2 = Metrics(json_list_test[0:1])
test_case3 = Metrics(json_list_test_NGCM_evap[:1])
combined = test_case1.merge(test_case2)

In [ ]:
combined.stats

In [ ]:
df_dict = combined.df_dict
var_list = combined.var_list
var_unit_list = combined.var_unit_list
var_ref_dict = combined.var_ref_dict
regions = combined.regions
stats = combined.stats

In [ ]:
print('var_list:', sorted(var_list))
print('var_unit_list:', sorted(var_unit_list))
print('var_ref_dict:', var_ref_dict)

In [ ]:
data_djf = df_dict['rms_xy']['djf']['global'][var_list].to_numpy()
data_mam = df_dict['rms_xy']['mam']['global'][var_list].to_numpy()
data_jja = df_dict['rms_xy']['jja']['global'][var_list].to_numpy()
data_son = df_dict['rms_xy']['son']['global'][var_list].to_numpy()
model_names = df_dict['rms_xyt']['ann']['global']['model'].tolist()
data_all = np.stack([data_djf, data_mam, data_jja, data_son])
print('data.shape:', data_all.shape)
print('len(var_list): ', len(var_list))
print('len(model_names): ', len(model_names))

xaxis_labels = var_list
yaxis_labels = model_names

In [ ]:
data_djf_nor = normalize_by_median(data_djf)
data_mam_nor = normalize_by_median(data_mam)
data_jja_nor = normalize_by_median(data_jja)
data_son_nor = normalize_by_median(data_son)

In [ ]:
data_all_nor = np.stack([data_djf_nor, data_mam_nor, data_jja_nor, data_son_nor])
data_all_nor.shape

In [ ]:
combined_all = library.merge(combined)
# combined_all = library.merge(test_case2)

# select models

In [ ]:
selected_models = ['ACCESS-ESM1-5', 'BCC-CSM2-MR', 'CAMS-CSM1-0', 'CESM2', 'CanESM5', 'CIESM', 'CMCC-CM2-SR5', 'E3SM-1-0', 'EC-Earth3',
                   'FGOALS-g3', 'FIO-ESM-2-0', 'GFDL-CM4', 'INM-CM5-0', 'IPSL-CM6A-LR', 'KACE-1-0-G', 'MIROC6', 'MRI-ESM2-0', 'MRI-ESM2-0', 'NorESM2-LM', 'NESM3', 
                   'SAM0-UNICON', 'TaiESM1', 'NeuralGCM-PCMDI', 'NeuralGCM-evap', 'ACE2-PCMDI']
#selected_models = ["ACCESS", "BCC-CSM", "CanESM", "CESM", "CMCC", "CNRM", "FGOALS", "GFDL", "GISS", "HadGEM", "INM", "IPSL", "MIROC", "MPI", "MRI-ESM", "NorESM"]


def selected_models_only(df, selected_models):
    # Selected models only
    model_names = df['model'].tolist()
    # print(model_names)
    for model_name in model_names:
        drop_model = True
        for keyword in selected_models:
            if keyword == model_name:
                drop_model = False
                # print(model_name, ' ', keyword)
                break
        if drop_model:
            # print(model_name)
            df.drop(df.loc[df['model']==model_name].index, inplace=True)
        df.reset_index(drop=True, inplace=True)
    return df

# Temperature and psl

In [ ]:
test_case1 = Metrics(json_list_test_NGCM[:])
combined_all = library.merge(test_case1)
test_case2 = Metrics(json_list_test[:])
combined_all = combined_all.merge(test_case2)
test_case_evap = Metrics(json_list_test_NGCM_evap)
combined_all = combined_all.merge(test_case_evap)

'''seasons = ['djf', 'mam', 'jja', 'son']
for season in seasons:
    for stat in ['rms_xy', 'mae_xy']:
        for region in ['global', 'NHEX', 'SHEX']:
            combined_all.df_dict[stat][season][region] = selected_models_only(combined_all.df_dict[stat][season][region], selected_models)'''

In [ ]:
combined_all.df_dict['rms_xy']['djf'].keys()

In [ ]:
combined_all.df_dict['rms_xy']['djf']

In [ ]:
var_list = ['ta-850', 'ta-200', 'ua-850', 'ua-200', 'va-850', 'va-200', 'zg-500', 'psl', 'pr']
# var_list = ['ta-850']

In [ ]:
combined_all.df_dict.keys()

In [ ]:
num_models = combined_all.df_dict['rms_xy']['djf']['global'].shape[0]
print(num_models)

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from pcmdi_metrics.graphics import portrait_plot

landscape = True
seasons = ['djf', 'mam', 'jja', 'son']

def plot_mean_climate(region, stat, save=None):
    print(f"Region is {region}, stat is {stat}")
    data = dict()
    
    for season in seasons:
        if landscape:
            data[season] = combined_all.df_dict[stat][season][region][var_list].to_numpy().T
        else:
            data[season] = combined_all.df_dict[stat][season][region][var_list].to_numpy()
    data_all = np.stack([data['djf'], data['mam'], data['jja'], data['son']])
    model_names = combined_all.df_dict[stat][season][region]['model'].tolist()
    
    print('data.shape:', data_all.shape)
    print('len(var_list): ', len(var_list))
    print('len(model_names): ', len(model_names))
    if landscape:
        yaxis_labels = var_list
        xaxis_labels = model_names
    else:
        xaxis_labels = var_list
        yaxis_labels = model_names
    if landscape:
        axis = 1
    else:
        axis = 0
    
    from pcmdi_metrics.graphics import normalize_by_median
    
    data_models_djf = combined_all.df_dict[stat]['djf'][region][~combined_all.df_dict[stat]['djf'][region]['model'].isin(['CMIP5 mean', 'CMIP6 mean'])][var_list].to_numpy()
    data_models_mam = combined_all.df_dict[stat]['mam'][region][~combined_all.df_dict[stat]['mam'][region]['model'].isin(['CMIP5 mean', 'CMIP6 mean'])][var_list].to_numpy()
    data_models_jja = combined_all.df_dict[stat]['jja'][region][~combined_all.df_dict[stat]['jja'][region]['model'].isin(['CMIP5 mean', 'CMIP6 mean'])][var_list].to_numpy()
    data_models_son = combined_all.df_dict[stat]['son'][region][~combined_all.df_dict[stat]['son'][region]['model'].isin(['CMIP5 mean', 'CMIP6 mean'])][var_list].to_numpy()
    # CMIP6:
    cmip6_models_djf = library.df_dict[stat]['djf'][region][~library.df_dict[stat]['djf'][region]['model'].isin(['CMIP5 mean', 'CMIP6 mean'])][var_list].to_numpy()
    cmip6_models_mam = library.df_dict[stat]['mam'][region][~library.df_dict[stat]['mam'][region]['model'].isin(['CMIP5 mean', 'CMIP6 mean'])][var_list].to_numpy()
    cmip6_models_jja = library.df_dict[stat]['jja'][region][~library.df_dict[stat]['jja'][region]['model'].isin(['CMIP5 mean', 'CMIP6 mean'])][var_list].to_numpy()
    cmip6_models_son = library.df_dict[stat]['son'][region][~library.df_dict[stat]['son'][region]['model'].isin(['CMIP5 mean', 'CMIP6 mean'])][var_list].to_numpy()
    if landscape:
        data_models_djf = data_models_djf.T
        data_models_mam = data_models_mam.T
        data_models_jja = data_models_jja.T
        data_models_son = data_models_son.T

        cmip6_models_son = cmip6_models_son.T
        cmip6_models_djf = cmip6_models_djf.T
        cmip6_models_mam = cmip6_models_mam.T
        cmip6_models_jja = cmip6_models_jja.T
        
    '''
    data_djf_nor = normalize_by_median(data['djf'], data_median=cmip6_models_djf, axis=axis)
    data_mam_nor = normalize_by_median(data['mam'], data_median=cmip6_models_mam, axis=axis)
    data_jja_nor = normalize_by_median(data['jja'], data_median=cmip6_models_jja, axis=axis)
    data_son_nor = normalize_by_median(data['son'], data_median=cmip6_models_son, axis=axis)
    '''
    
    data_djf_nor = normalize_by_median(data['djf'], data_median=data_models_djf, axis=axis)
    data_mam_nor = normalize_by_median(data['mam'], data_median=data_models_mam, axis=axis)
    data_jja_nor = normalize_by_median(data['jja'], data_median=data_models_jja, axis=axis)
    data_son_nor = normalize_by_median(data['son'], data_median=data_models_son, axis=axis)
    
    data_all_nor = np.stack([data_djf_nor, data_mam_nor, data_jja_nor, data_son_nor])
    print(data_all_nor.shape)
    if landscape:
        figsize = (35, 10)
    else:
        #figsize = (7, 15)
        figsize = (13, 15)
    
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(121, anchor='NW')
    
    if not landscape:
        ax2 = fig.add_subplot(122, anchor='NW')
        xaxis_tick_labels_top_and_bottom = True
        borderpad = -6
        legend_box_xy = (0, -0.25)
        legend_box_size = 8
        legend_fontsize = 15
        data_to_plot = data_all_nor
    else:
        xaxis_tick_labels_top_and_bottom = False
        borderpad = -2
        legend_box_xy = (-0.08, 1.3)
        legend_box_size = 2
        legend_fontsize = 10
        data_to_plot = data_all_nor
        
    axins = inset_axes(ax,
                        width="100%",  
                        height="2.5%",
                        loc='lower center',
                        borderpad=borderpad
                       )
    if stat=='rms_xy':
        stat_name = 'RMSE'
    elif stat=='mae_xy':
        stat_name = 'MAE'
    fig, ax, cbar = portrait_plot(data_all_nor,
                                  xaxis_labels=xaxis_labels, 
                                  yaxis_labels=yaxis_labels, 
                                  cbar_label=stat_name+', normalized by median',
                                  fig=fig,
                                  ax=ax,
                                  box_as_square=True,
                                  #box_as_square=False,
                                  vrange=(-0.5, 0.5),
                                  figsize=figsize,
                                  xaxis_fontsize=12,
                                  yaxis_fontsize=13,
                                  xaxis_tick_labels_top_and_bottom=xaxis_tick_labels_top_and_bottom,
                                  xticklabel_rotation=60,
                                  cmap='RdYlBu_r',
                                  cmap_bounds=[-0.5, -0.4, -0.3, -0.2, -0.1, 0, 0.1, 0.2, 0.3, 0.4, 0.5],
                                  cbar_kw={"extend": "both",
                                           #aspect": 100,
                                           "orientation": 'horizontal',
                                           #"location": 'bottom',
                                           #"pad": 0.001,
                                           "shrink": 0.8,
                                           #"anchor": (0.5, 0.9),
                                           #"use_gridspec": False,
                                           "cax": axins,
                                          },
                                  colorbar_off=False,
                                  #missing_color='white',
                                  missing_color='lightgrey',
                                  legend_on=True,
                                  legend_labels=['DJF', 'MAM', 'JJA', 'SON'],
                                  legend_box_xy=legend_box_xy,
                                  legend_box_size=legend_box_size,
                                  legend_lw=1,
                                  legend_fontsize=legend_fontsize,
                                  #logo_rect = [0.85, 0.15, 0.07, 0.07],
                                  logo_off=True,
                                  inner_line_color="white",
                                  inner_line_width=0.1,
                                 )
    
    if landscape:
        # change font color for labels
        for i in range(0, num_models-3):
            plt.setp(ax.get_xticklabels()[i], color='blue')
        for i in range(num_models-3, num_models):
            plt.setp(ax.get_xticklabels()[i], color='red')
        ax.axvline(x=num_models-3, color='k', linewidth=3)
        ax.axvline(x=num_models, color='k', linewidth=3)
    plt.tight_layout()
    if save:
        plt.savefig(save, bbox_inches='tight', dpi=200)
    plt.show()

In [ ]:
plot_mean_climate(region='global', stat='rms_xy', save='potrait_mean_climate_global_rmse.png')
plot_mean_climate(region='global', stat='mae_xy', save='potrait_mean_climate_global_mae.png')

In [ ]:
plot_mean_climate(region='SHEX', stat='rms_xy')
plot_mean_climate(region='SHEX', stat='mae_xy')


In [ ]:
plot_mean_climate(region='TROPICS', stat='rms_xy')
plot_mean_climate(region='TROPICS', stat='mae_xy')

In [ ]:
var_ref_dict = combined_all.var_ref_dict

In [ ]:
fig = plt.figure(figsize=(8, 8))
var = "ta-850"
season = "djf"
region = "global"
stddev = combined_all.df_dict["std_xy"][season][region][var].to_numpy()
refstd = combined_all.df_dict['std-obs_xy'][season][region][var][0]
corrcoeff = combined_all.df_dict["cor_xy"][season][region][var].to_numpy()
models = combined_all.df_dict["cor_xy"][season][region]['model'].to_list()
# Customize plot to highlight user's model in red, others in grey
colors = list()
labels = list()
markers = list()
markersizes = list()
zorders = list()
for i, model in enumerate(models):
    if i == len(models)-1:
        labels.append(model)
        colors.append('red')
        markers.append('o')
        markersizes.append(12)
        zorders.append(100)
    elif i == len(models)-2:
        labels.append(model)
        colors.append('blue')
        markers.append('o')
        markersizes.append(12)
        zorders.append(100)
    elif i == len(models)-3:
        labels.append(model)
        colors.append('pink')
        markers.append('o')
        markersizes.append(12)
        zorders.append(100)
    else:
        if i == 0:
            labels.append('CMIP6 models')
        else:
            labels.append(None)
        colors.append('grey')
        markersizes.append(8)
        markers.append('o')
        zorders.append(10)

fig, ax = TaylorDiagram(stddev, corrcoeff, refstd, fig, colors=colors, normalize=True,
                        labels=labels, markers=markers,
                        markersizes=markersizes, zorders=zorders, ref_label='Ref: '+var_ref_dict[var])

ax.legend(bbox_to_anchor=(1.05, 1.05), loc='upper right', ncol=1)
fig.suptitle(', '.join([var, season, region]), fontsize=20)

In [ ]:
var_ref_dict